In [1]:
import pandas as pd
import os

In [2]:
df = pd.DataFrame()

for file_name in os.listdir("./Raw Dataset"):
    if file_name.startswith("mmlu"):
        print("file path:", os.path.join("./Raw Dataset", file_name))
        tmp_df = pd.read_json(os.path.join("./Raw Dataset", file_name))
        df = pd.concat([df, tmp_df])

print(df.shape)

file path: ./Raw Dataset\mmlu_0_6000.json
file path: ./Raw Dataset\mmlu_12000_last.json
file path: ./Raw Dataset\mmlu_6000_12000.json
(17644, 7)


In [3]:
df.head()

,prompt,context,A,B,C,D,E
0,A leading question is LEAST likely to be permi...,Leading questions may also be permitted on dir...,asked on cross-examination of an expert witness.,,related to preliminary matters such as the nam...,asked on direct examination of a young child.,asked on direct examination of a disinterested...
1,The strongest constitutional basis for the ena...,The federal government's role in higher educat...,power to tax and spend for the general welfare.,police power.,power to enforce the privileges and immunities...,,war and defense power.
2,In which of the following cases is a convictio...,The defendant.* Stone's past criminal record i...,Hayes unbuttoned the vest of a man too drunk t...,Having induced a woman to enter his hotel room...,A confederate of Brown pushed a man in order t...,"Johnson forced his way into a woman's home, bo...",
3,National regulation of predatory wild animals ...,The state of New Mexico challenged the federal...,"unconstitutional, because regulation and contr...","constitutional, because Congress has authority...",,"unconstitutional, because wild animals as defi...","constitutional, because the protection of wild..."
4,In which of the following situations is Defend...,The Commonwealth charged larceny for all the m...,Police arrested Thief and recovered goods he h...,Defendant misrepresented his identity to secur...,,Believing that state law made it a crime to pu...,"Defendant, intending to kill Selma, shot at Se..."


In [4]:
df.rename(columns={"top_best_answer": "answer"}, inplace=True)
df.head()

,prompt,context,A,B,C,D,E
0,A leading question is LEAST likely to be permi...,Leading questions may also be permitted on dir...,asked on cross-examination of an expert witness.,,related to preliminary matters such as the nam...,asked on direct examination of a young child.,asked on direct examination of a disinterested...
1,The strongest constitutional basis for the ena...,The federal government's role in higher educat...,power to tax and spend for the general welfare.,police power.,power to enforce the privileges and immunities...,,war and defense power.
2,In which of the following cases is a convictio...,The defendant.* Stone's past criminal record i...,Hayes unbuttoned the vest of a man too drunk t...,Having induced a woman to enter his hotel room...,A confederate of Brown pushed a man in order t...,"Johnson forced his way into a woman's home, bo...",
3,National regulation of predatory wild animals ...,The state of New Mexico challenged the federal...,"unconstitutional, because regulation and contr...","constitutional, because Congress has authority...",,"unconstitutional, because wild animals as defi...","constitutional, because the protection of wild..."
4,In which of the following situations is Defend...,The Commonwealth charged larceny for all the m...,Police arrested Thief and recovered goods he h...,Defendant misrepresented his identity to secur...,,Believing that state law made it a crime to pu...,"Defendant, intending to kill Selma, shot at Se..."


In [5]:
df_origin = pd.read_json("./Raw Dataset/17k mmlu.json")
df_origin.head()

,prompt,context,A,B,C,D,E,answer
0,A leading question is LEAST likely to be permi...,Leading questions may also be permitted on dir...,asked on cross-examination of an expert witness.,,related to preliminary matters such as the nam...,asked on direct examination of a young child.,asked on direct examination of a disinterested...,E
1,The strongest constitutional basis for the ena...,Some well-to-do colleges have need-based aid o...,power to tax and spend for the general welfare.,police power.,power to enforce the privileges and immunities...,,war and defense power.,A
2,In which of the following cases is a convictio...,Aggravated Robbery 4-15 years in prison. ===In...,Hayes unbuttoned the vest of a man too drunk t...,Having induced a woman to enter his hotel room...,A confederate of Brown pushed a man in order t...,"Johnson forced his way into a woman's home, bo...",,A
3,National regulation of predatory wild animals ...,If the wild animal species are not kept in che...,"unconstitutional, because regulation and contr...","constitutional, because Congress has authority...",,"unconstitutional, because wild animals as defi...","constitutional, because the protection of wild...",B
4,In which of the following situations is Defend...,The prosecutor may decide not to prosecute a c...,Police arrested Thief and recovered goods he h...,Defendant misrepresented his identity to secur...,,Believing that state law made it a crime to pu...,"Defendant, intending to kill Selma, shot at Se...",D


In [6]:
answers = df_origin["answer"]
df["answer"] = answers

In [7]:
df["answer"].value_counts()

answer
C    3698
A    3651
B    3503
D    3402
E    3390
Name: count, dtype: int64

In [8]:
df_with_nan = df[df.isna().any(axis=1)]
print(df_with_nan.shape)

(0, 8)


In [9]:
import regex as re

def clean_string(s):
    text = re.sub(r'#+', '', s)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'\(\)', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def use_sep(s):
    text = re.sub(r'#####', '####', s)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'\(\)', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [10]:
df["context"] = df["context"].apply(clean_string)
df.to_json("./Clean Dataset/17k mmlu no #.json", orient="records")